In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords, inaugural
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string

The `remove_noise`, `get_all_words`, `get_tweets_for_model`, and `build_sentiment_model` functions are all reusued or modified versions of the functions found in the code from this website:
https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk

In [2]:
#The data we are going to be looking at is going to be rife with all the quirks of modern internet speak, so this function
#is to normalize it into something more akin to formal writing.
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []
    #Loop to remove hyperlinks, mentions, and other special characters. 
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        
        #Assign part of speech as part of a pair.
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        
        #Lemmatization converts words to their canonical form.
        #This is so words like "run", "ran", and "running" are all treated as the same word.
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        #Append and return our cleaned tokens (words)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [3]:
#A generator function that takes a list of tweets as an argument to provide a list of words in all of 
#the tweet tokens joined.
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [4]:
#Convert the tweets from a list of cleaned tokens to dictionaries with keys as the tokens and True as values. 
def get_tweets_for_model(cleaned_tokens_list):
    #The corresponding dictionaries are stored in positive_tokens_for_model and negative_tokens_for_model.
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [5]:
#The function creates a model that classifies a sentence whether it is positive or negative.
#Trained on Twitter data as it is the only labeled data.
def build_sentiment_model():
    
    #Load the required data.
    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]
    
    #Load in stopwords so we can filter them out.
    stop_words = stopwords.words('english')
    
    #Tokenize all the tweets as items so we can clean them.
    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
    
    #Initialize the lists to add cleaned tweets to later.
    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []
    
    #Clean the tokenized tweets by calling the remove_noise function.
    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
        
    #These two functions are used in tandem to find the frequencies of the most used positive words.
    all_pos_words = get_all_words(positive_cleaned_tokens_list)
    freq_dist_pos = FreqDist(all_pos_words)

    #Store the cleaned tokens into datasets.
    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
    positive_dataset = [(tweet_dict, "Positive") for tweet_dict in positive_tokens_for_model]
    negative_dataset = [(tweet_dict, "Negative") for tweet_dict in negative_tokens_for_model]
    dataset = positive_dataset + negative_dataset
    
    #Train the model and return the classifier.
    train_data = dataset[:7000]
    test_data = dataset[7000:]
    classifier = NaiveBayesClassifier.train(train_data)
    
    return classifier

In [6]:
#Using the classifier, parse sentences in transcript and determine positive or negative
#Return ratio of whole text
def positivity_score(transcript, classifier):
    positive_sentences = 0
    negative_sentences = 0
    #Loop through each sentence, split up as tokens.
    for sent in sent_tokenize(transcript):
        #Collect the tokens for that sentence and remove noise.
        custom_tokens = remove_noise(word_tokenize(sent))
        #Run the sentence through the classifier.
        #Depending on the sentiment, we update the corresponding counting variable.
        if(classifier.classify(dict([token, True] for token in custom_tokens)) == 'Positive'):
            positive_sentences += 1
        else:
            negative_sentences += 1
    #Return a ratio of positive sentences over the total.
    return positive_sentences/(positive_sentences+negative_sentences)  